In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import platform
from matplotlib.ticker import FuncFormatter
import plotly.graph_objects as go
from matplotlib import rcParams
import csv
import matplotlib.ticker as ticker
from plotly.subplots import make_subplots
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
import plotly.express as px

plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows 기본 한글 폰트
plt.rcParams['axes.unicode_minus'] = False     # 마이너스 기호 깨짐 방지


total22_df = pd.read_csv("전연령22_utf8.csv", low_memory=False)
senior22_df = pd.read_csv("노인승하차22_utf8.csv", low_memory=False)
total24_df = pd.read_csv("전연령24_utf8.csv", low_memory=False)
senior24_df = pd.read_csv("노인승하차24_utf8.csv", low_memory=False)
total23_df = pd.read_csv("전연령23_utf8.csv", low_memory=False)
senior23_df = pd.read_csv("노인승하차23_utf8.csv", low_memory=False)
#senior_care_df = pd.read_csv("senior_care_utf8.csv", low_memory=False) # 파일이 사라짐
sme_df = pd.read_csv("sme.csv", low_memory=False)
park_df = pd.read_csv("TB_PTP_PRK_M.csv", low_memory=False)
safety_df = pd.read_csv("안전사고_utf.csv", low_memory=False)
elderly_2020_2022 = pd.read_csv(
    "elderly2020_2022.csv",
    low_memory=False,
    quoting=csv.QUOTE_NONE,
    encoding="utf-8",
    on_bad_lines='skip'  # 문제 있는 라인은 건너뜀
)
elderly_2023_2025 = pd.read_csv(
    "elderly2023_2025.csv",
    low_memory=False,
    quoting=csv.QUOTE_NONE,
    encoding="utf-8",
    on_bad_lines='skip'
)
weather_raw = pd.read_csv("weath20_25_utf8.csv", encoding='utf-8')
rain_raw = pd.read_csv("rain20_25_utf8.csv", encoding='utf-8') # 강수량 파일명 확인 필요
import pandas as pd

# ----------------------------------------------------------
# [1] 주요 노인 이용 역 15곳 선정 및 데이터 입력
# ----------------------------------------------------------
# 선정 기준: 승하차 인원 상위권 + 사고 다발 우려 지역
data_expanded = {
    '역명': [
        '제기동', '청량리', '종로3가', '종로5가', '동묘앞',   # 1. 도심/시장 라인 (전통적 핫플레이스)
        '연신내', '수유', '미아사거리', '쌍문', '창동',     # 2. 강북 주거 벨트 (출발지 수요 많음)
        '신도림', '구로디지털단지', '신림', '사당',         # 3. 서남권 거점 (환승/주거)
        '고속터미널'                                      # 4. 대형 환승 허브 (복잡도 높음)
    ],
    '호선': [
        '1호선', '1호선', '1호선', '1호선', '1호선',
        '3호선', '4호선', '4호선', '4호선', '4호선',
        '2호선', '2호선', '2호선', '2호선',
        '3호선'
    ],
    # 엘리베이터 (E/V): 노인 이동의 필수 수단 (적을수록 위험)
    'EV_개수': [
        2, 4, 3, 2, 2,    # 도심권은 구조상 설치가 어려워 적은 편
        4, 3, 3, 2, 3,    # 주거지도 이용객 대비 부족
        4, 3, 3, 5,       # 유동인구가 워낙 많은 곳들
        3
    ],
    # 에스컬레이터 (E/S): 사고가 가장 많이 나는 시설
    'ES_개수': [
        6, 8, 12, 4, 6,
        16, 8, 8, 6, 8,
        14, 10, 8, 18,
        24                # 고터 등 환승역은 E/S가 매우 많음 (그만큼 사고 위험도 높음)
    ],
    # 휠체어리프트 (W/L): 엘리베이터 없는 곳의 대체 수단 (위험함)
    'WL_개수': [
        1, 2, 3, 1, 2,
        0, 1, 1, 2, 1,
        2, 1, 1, 2,
        1
    ]
}

# ----------------------------------------------------------
# [2] 데이터프레임 생성 및 저장
# ----------------------------------------------------------
df_facility_exp = pd.DataFrame(data_expanded)

# CSV 파일로 저장 (한글 깨짐 방지)
file_name = "station_facilities_expanded.csv"
df_facility_exp.to_csv(file_name, index=False, encoding='utf-8-sig')

print(f"✅ 확장된 편의시설 데이터 파일 '{file_name}' 생성 완료!")
print(f"   - 분석 대상 역: 총 {len(df_facility_exp)}개")
print("-" * 50)
print(df_facility_exp)


✅ 확장된 편의시설 데이터 파일 'station_facilities_expanded.csv' 생성 완료!
   - 분석 대상 역: 총 15개
--------------------------------------------------
         역명   호선  EV_개수  ES_개수  WL_개수
0       제기동  1호선      2      6      1
1       청량리  1호선      4      8      2
2      종로3가  1호선      3     12      3
3      종로5가  1호선      2      4      1
4       동묘앞  1호선      2      6      2
5       연신내  3호선      4     16      0
6        수유  4호선      3      8      1
7     미아사거리  4호선      3      8      1
8        쌍문  4호선      2      6      2
9        창동  4호선      3      8      1
10      신도림  2호선      4     14      2
11  구로디지털단지  2호선      3     10      1
12       신림  2호선      3      8      1
13       사당  2호선      5     18      2
14    고속터미널  3호선      3     24      1


In [17]:

# ---------------------------------------------------------
# [1] 데이터 전처리
# ---------------------------------------------------------
df = safety_df.copy()

# 1. 시간 숫자 추출
df['시_임시'] = pd.to_numeric(
    df['발생시간'].astype(str).str.split(':').str[0], 
    errors='coerce'
).fillna(-1).astype(int)

# 2. 시간대 포맷 매핑
def map_to_subway_time(h):
    if h == -1: return '시간미상'
    if h < 6: return '06시간대이전'
    if h >= 24: return '24시간대이후'
    return f"{h:02d}-{h+1:02d}시간대"

df['시간대구분'] = df['시_임시'].apply(map_to_subway_time)

# 3. 시간대별 총합 집계 (사고유형 통합)
time_counts = df.groupby('시간대구분').size().reset_index(name='총건수')

# 4. 정렬 순서 지정
time_order = [
    '06시간대이전', '06-07시간대', '07-08시간대', '08-09시간대', '09-10시간대',
    '10-11시간대', '11-12시간대', '12-13시간대', '13-14시간대', '14-15시간대',
    '15-16시간대', '16-17시간대', '17-18시간대', '18-19시간대', '19-20시간대',
    '20-21시간대', '21-22시간대', '22-23시간대', '23-24시간대', '24시간대이후',
    '시간미상'
]

# ---------------------------------------------------------
# [2] 시각화 (폰트 확대 + 주석 제거)
# ---------------------------------------------------------
fig = go.Figure()

fig.add_trace(go.Bar(
    x=time_counts['시간대구분'],
    y=time_counts['총건수'],
    name='사고 건수',
    marker_color='#1565C0', # 진한 파란색 (가독성 좋음)
    # [폰트 확대] 막대 위 숫자
    text=time_counts['총건수'],
    textposition='outside', # 막대 위쪽으로 빼서 잘 보이게 함
    textfont=dict(size=16, color='black', family="Arial Black"), # 숫자 크기 16, 굵게
    hovertemplate='<b>%{x}</b><br>총 발생: %{y}건<extra></extra>'
))

# ---------------------------------------------------------
# [3] 레이아웃 설정 (전체 글자 크기 UP)
# ---------------------------------------------------------
fig.update_layout(
    # [폰트 확대] 제목
    title=dict(
        text="<b>시간대별 전체 안전사고 발생 현황</b>",
        x=0.5, y=0.95,
        font=dict(size=30, family="Malgun Gothic", color='#333') # 제목 30pt
    ),
    # [폰트 확대] X축
    xaxis=dict(
        title="시간대",
        categoryorder='array',
        categoryarray=time_order,
        tickangle=-45,
        tickfont=dict(size=16, weight='bold'),   # 눈금 16pt
        title_font=dict(size=20, weight='bold')  # 축 제목 20pt
    ),
    # [폰트 확대] Y축
    yaxis=dict(
        title="발생 건수 (건)",
        tickfont=dict(size=16),
        title_font=dict(size=20, weight='bold'),
        gridcolor='#f0f0f0',
        zeroline=False
    ),
    template="plotly_white",
    margin=dict(t=100, b=100), # 여백 확보
    height=650, # 그래프 높이 키움
    showlegend=False
)

fig.show()

In [15]:


# ---------------------------------------------------------
# [1] 데이터 전처리 (기존 로직 유지)
# ---------------------------------------------------------
df = safety_df.copy()

df['시'] = df['발생시간'].astype(str).str.split(':').str[0]
df['시'] = pd.to_numeric(df['시'], errors='coerce').fillna(-1).astype(int)

core_hours = [11, 12, 13, 14, 15, 16, 17]
duration_core = 7
duration_other = 17

def classify_time_simple(h):
    if h in core_hours: return '노인(7h)'
    else: return '그외(17h)'

df['시간대구분'] = df['시'].apply(classify_time_simple)
df['역명_검색용'] = df['발생역'].astype(str).str.split('(').str[0].str.replace('역$', '', regex=True).str.strip()

# ---------------------------------------------------------
# [2] 시각화 함수 (색상 복구 + 디자인 유지)
# ---------------------------------------------------------
def plot_density_vivid_design(station_list, title_text):
    # 1. 데이터 필터링
    target_data = df[df['역명_검색용'].isin(station_list)].copy()
    counts = target_data.groupby(['역명_검색용', '시간대구분', '사고유형']).size().reset_index(name='건수')
    
    # 2. 밀도 계산
    def calculate_density(row):
        if row['시간대구분'] == '노인(7h)':
            return row['건수'] / duration_core
        else:
            return row['건수'] / duration_other
    counts['밀도'] = counts.apply(calculate_density, axis=1)
    
    # 3. 시각화 준비
    fig = go.Figure()
    
    # ★ [색상 복구] 선명하고 구분이 확실한 컬러
    accident_colors = {
        '출입문관련': '#4A90E2', # 선명한 파랑
        '발빠짐': '#F5A623',    # 선명한 주황
        '승강설비관련': '#7ED321', # 선명한 초록
        '열차내 사고': '#BD10E0', # 선명한 보라
        '역구내 사고': '#D0021B', # 선명한 빨강
        '기타': '#9B9B9B'       # 회색
    }
    
    sorted_types = counts.groupby('사고유형')['밀도'].sum().sort_values(ascending=False).index.tolist()
    
    for accident_type in sorted_types:
        subset = counts[counts['사고유형'] == accident_type].copy()
        
        # 정렬
        subset['역명_검색용'] = pd.Categorical(subset['역명_검색용'], categories=station_list, ordered=True)
        subset = subset.sort_values(['역명_검색용', '시간대구분'], ascending=[True, False])
        
        fig.add_trace(go.Bar(
            x=[subset['역명_검색용'], subset['시간대구분']], 
            y=subset['밀도'],
            name=accident_type,
            marker_color=accident_colors.get(accident_type, '#9B9B9B'),
            text=subset['건수'],
            textposition='inside',
            # 텍스트 색상: 진한 배경이므로 흰색으로 변경
            textfont=dict(color='white', size=16, family="Arial Black"), 
            hovertemplate='<b>%{x}</b><br>유형: %{data.name}<br>밀도: %{y:.2f} (실제 %{text}건)<extra></extra>'
        ))

    # 4. 레이아웃 및 세로선 추가
    fig.update_layout(
        title=dict(
            text=f"<b>{title_text}</b>",
            x=0.5, y=0.95,
            font=dict(size=26, family="Malgun Gothic", color='#333')
        ),
        xaxis=dict(
            title="", 
            tickfont=dict(size=20, color='#333'),
            linecolor='#ddd'
        ),
        yaxis=dict(
            title="시간당 발생 건수 (건/Hour)",
            title_font=dict(size=18, weight='bold'),
            tickfont=dict(size=16),
            gridcolor='#f0f0f0',
            zeroline=False
        ),
        legend=dict(
            orientation="h",       
            yanchor="bottom", y=1.02, 
            xanchor="center", x=0.5,  
            title_text="",
            font=dict(size=16)
        ),
        barmode='stack',
        template="plotly_white",
        margin=dict(t=100, b=80),
        height=600
    )
    
    # ★ [핵심] 역마다 세로 구분선(Vertical Divider) 추가 ★
    # 역이 5개면 그룹이 5개. 각 그룹은 막대가 2개(노인, 그외)씩 있음.
    # Plotly 카테고리 축 좌표는 0, 1, 2, ... 순서임.
    # 첫 번째 그룹(0, 1)과 두 번째 그룹(2, 3) 사이의 좌표는 1.5
    # 따라서 1.5, 3.5, 5.5, 7.5 위치에 선을 그으면 됨.
    for x_pos in [1.5, 3.5, 5.5, 7.5]:
        fig.add_vline(x=x_pos, line_width=2, line_dash="dash", line_color="#555")

    # 주석
    fig.add_annotation(
        text="※ 점선: 역 구분선 / 막대 높이: 사고 밀도 / 내부 숫자: 실제 건수",
        xref="paper", yref="paper",
        x=1.0, y=1.2, 
        showarrow=False,
        font=dict(size=14, color="gray", weight='bold'),
        align="right"
    )

    fig.show()

# ---------------------------------------------------------
# [3] 실행
# ---------------------------------------------------------

# 1. 승차 집중 역 (주거지)
boarding_stations = ['연신내', '쌍문', '창동', '까치산', '수유']
print("📊 1. 승차 집중 역 (주거지) 분석")
plot_density_vivid_design(boarding_stations, "[승차 Top 5] 시간당 사고 밀도 및 원인")

# 2. 하차 집중 역 (목적지)
alighting_stations = ['종로3가', '제기동', '청량리', '종로5가', '회현']
print("📊 2. 하차 집중 역 (목적지) 분석")
plot_density_vivid_design(alighting_stations, "[하차 Top 5] 시간당 사고 밀도 및 원인")

📊 1. 승차 집중 역 (주거지) 분석


📊 2. 하차 집중 역 (목적지) 분석


In [26]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# ---------------------------------------------------------
# 1. 데이터 준비 (사용자 제공 데이터 로직 유지)
# ---------------------------------------------------------
data = {
    'Time': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23],
    # 예시 데이터 (실제 데이터가 있다면 교체하세요)
    'Accident_Count': [
        2, 5, 15, 20, 10, 5,    # 오전 (그 외)
        12, 14, 15, 13, 14, 12, # 11~16시 (노인 주활동)
        22, 25, 15, 8, 5, 3, 2  # 오후/저녁 (그 외)
    ]
}
df = pd.DataFrame(data)

# (1) 시간대 구분 함수
def classify_time(hour):
    if 11 <= hour < 17:  
        return '노인 주 활동 시간\n(11시~17시)'
    else:
        return '그 외 시간대\n(출퇴근 및 기타)'

# (2) 그룹 생성 및 집계
df['Time_Group'] = df['Time'].apply(classify_time)
grouped_counts = df.groupby('Time_Group')['Accident_Count'].sum().reset_index()
duration = df.groupby('Time_Group')['Time'].count().reset_index()
duration.columns = ['Time_Group', 'Duration_Hour']

# (3) 밀도 계산
result = pd.merge(grouped_counts, duration, on='Time_Group')
result['Density'] = result['Accident_Count'] / result['Duration_Hour']

# ---------------------------------------------------------
# 3. 시각화 (PPT 최적화 - 폰트 확대 & 디자인)
# ---------------------------------------------------------

# 색상 설정 (강조색: 노인 시간)
color_map = {
    '노인 주 활동 시간\n(11시~17시)': '#E63946', # 잘 보이는 빨강
    '그 외 시간대\n(출퇴근 및 기타)': '#1D3557'  # 차분한 남색
}

fig = px.bar(
    result,
    x='Density',
    y='Time_Group',
    color='Time_Group',
    orientation='h', # 수평 막대
    text='Density',  # 막대 끝 텍스트 기준
    color_discrete_map=color_map,
    # 순서 고정 (노인 시간이 위로 오게 하려면 리스트 순서 조정)
    category_orders={'Time_Group': ['노인 주 활동 시간\n(11시~17시)', '그 외 시간대\n(출퇴근 및 기타)']}
)

# [핵심] 폰트 및 스타일 대폭 강화
fig.update_traces(
    texttemplate='%{x:.2f} 건/h', # 소수점 2자리 + 단위
    textposition='outside',       # 막대 바깥쪽 표시
    textfont=dict(size=20, family="Arial Black", color="black"), # 데이터 라벨 폰트 (매우 큼)
    cliponaxis=False # 텍스트가 그래프 밖으로 나가도 잘리지 않게 함
)

fig.update_layout(
    showlegend=False, # 범례 숨김 (축에 이미 표시됨)
    title={
        'text': "<b>시간대별 지하철 안전사고 발생 밀도 비교</b>",
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 28, 'family': 'Malgun Gothic', 'color': '#333'} # 제목 폰트 28pt
    },
    xaxis=dict(
        title="시간당 사고 발생 건수 (건/Hour)",
        title_font=dict(size=20, weight='bold'), # X축 제목 20pt
        tickfont=dict(size=16),                  # X축 눈금 16pt
        range=[0, result['Density'].max() * 1.4], # 오른쪽 여백 충분히 확보 (텍스트 안 잘리게)
        gridcolor='#eee',
        showline=True, linewidth=2, linecolor='black' # 축 라인 진하게
    ),
    yaxis=dict(
        title="", # Y축 제목 생략 (직관적임)
        tickfont=dict(size=20, weight='bold', color='#333'), # Y축 라벨(시간대 이름) 20pt
        showline=False
    ),
    font=dict(family="Malgun Gothic"), # 전체 기본 폰트
    plot_bgcolor='white', # 배경 흰색
    margin=dict(l=50, r=100, t=100, b=50), # 여백 조정
    height=550 # 높이 설정
)

# 그래프 출력
fig.show()

print("=== 분석 결과 (밀도) ===")
print(result[['Time_Group', 'Density']].round(2))

=== 분석 결과 (밀도) ===
              Time_Group  Density
0    그 외 시간대\n(출퇴근 및 기타)    10.54
1  노인 주 활동 시간\n(11시~17시)    13.33
